# Two-stage approach + R_convexity

- In this notebook, I implemented the two-stage approach and incorporated Tim's R_convexity to calculate the KLLN model.
- I modified the model so that, under the two-stage approach, liquidity-unconstrained agents make deposits.
- This results in changes to the money demand function and aggregate deposit sections.
- Specifically, liquidity-unconstrained buyers now receive a marginal benefit of i, and the total deposits increase by the amount of idle cash held by unconstrained buyers.
- The code was adjusted to account for R_convexity, with a focus on using Rfit within the R() function. Additionally, to calculate dJ (or dF), Rfit.derivative()(rho) is utilized.

Other critical areas to check include adjustments considering the measure of active and non-active buyers:
- Money demand function
- Equilibrium outcome
- Loan/deposit market clearing condition

Please review these aspects carefully.

Additionally, I still feel that the modifications to the money demand function across the z_range may not be fully complete. In particular, it’s uncertain whether i_bound, among other factors, should also be considered. I’ll finalize this part once Tim completes the Lemma arrangements.

## SME: BJ goods and competitive banks 
Given policy $\tau$, we get real money balance $(z)$ and market interest rate $(i=i_{l}=i_{d})$ by solving

\begin{equation}
\begin{split}
z^{\star} \leftarrow  \frac{1+\tau -\beta}{\beta} &= R(z,i)
\\
i_{l}=i_{d}=i^{\star} \leftarrow (1-n) \cdot z + n \cdot \int^{\overline{\rho}}_{\hat{\rho}} [\alpha_{1}+2\alpha_{2}(1-J(\rho))] \delta(i,\rho,z)\text{d}J(\rho)&= n \cdot \int^{\tilde{\rho}}_{\underline{\rho}} [\alpha_{1}+2\alpha_{2}(1-J(\rho))] \xi(i,\rho,z) \text{d}J(\rho)
\end{split}
\label{eq:k-phi-high-type-case1}
\end{equation}

where
\begin{equation*}
\begin{split}
R(z,i) &:=  \color{blue}{(1-n)} \cdot i + \color{blue}{n}\bigg[\int^{\tilde{\rho}}_{\underline{\rho}} i [\alpha_{1}+2\alpha_{2}(1-J(\rho))] \text{d}J(\rho)
\\
&+ \int^{\hat{\rho}}_{\tilde{\rho}} [\alpha_{1}+2\alpha_{2}(1-J(\rho))] \bigg[\frac{1}{\rho}\bigg(\frac{z}{\rho}\bigg)^{-\sigma} -1 \bigg] \text{d}J(\rho)
\\
&+ \int^{\overline{\rho}}_{\hat{\rho}} [\alpha_{1}+2\alpha_{2}(1-J(\rho))] \cdot i \text{d}J(\rho)\bigg]
\end{split}
\end{equation*}

* Two equations and two unknowns 

In [1]:
import bcw_bj as main
import hlmw as hlmw
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from scipy import interpolate
from scipy.optimize import brentq, fsolve
#import tikzplotlib
import time

In [2]:
plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'serif'

In [3]:
font = {'family' : 'serif','weight':'normal',
        'size'   : 10}
plt.rc('font', **font)

In [4]:
σ_DM=0.4
Ubar_CM=1.9
n=0.65
α_1=0.1
β=0.9814094266351937

In [5]:
model = main.baseline_mod(σ_DM=σ_DM, Ubar_CM=Ubar_CM, n=n,τ_max = 0.00,α_1=α_1,β=β,τgrid_size = 10,N_reimann = 300,Tol=1e-10)

model_hlmw =hlmw.hlmw_mod(σ_DM=σ_DM, Ubar_CM=Ubar_CM, n=n,τ_max = 0.00,α_1=α_1,β=β,τgrid_size = 10,N_reimann = 300,Tol=1e-10)

### SME: HLMW + perfectly competitive banks (Baseline)

* Vary $\tau \in [\beta-1, 0.1]$

In [ ]:
z_guess = 0.6
i_guess = 0.01
result = model.SME_stat(z_guess, i_guess) # this uses fsolve

-0.018590573364806295


In [ ]:
z = result['allocation_grid']['zstar']
q = result['allocation_grid']['qstar']
ξ = result['allocation_grid']['ξstar']
i = result['allocation_grid']['istar']
DM_surplus = result['allocation_grid']['DM_surplus']
W = result['allocation_grid']['w_star']
W_BCW = result['allocation_grid']['w_bcw']

mpy = result['stat_grid']['mpy_star']
markup = result['stat_grid']['markup_mean']
markup_cv = result['stat_grid']['markup_cv']
loan_gdp = result['stat_grid']['credit_gdp']
loan_gdp_bcw = result['stat_grid']['credit_gdp_bcw']

### HLMW: without banks

In [ ]:
result_hlmw = model_hlmw.SME_stat() # this uses brentq only

In [ ]:
z_hlmw = result_hlmw['allocation_grid']['zstar']
q_hlmw = result_hlmw['allocation_grid']['qstar']
W_hlmw = result_hlmw['allocation_grid']['w_star']
DM_surplus_hlmw = result_hlmw['allocation_grid']['DM_surplus']

mpy_hlmw = result_hlmw['stat_grid']['mpy_star']
markup_hlmw = result_hlmw['stat_grid']['markup_mean']
markup_cv_hlmw = result_hlmw['stat_grid']['markup_cv']

In [ ]:
τ_grid = model.τ_grid # inflation rate grid
i_grid = model.i_policy(τ_grid)

In [ ]:
font = {'family' : 'serif','weight':'normal',
        'size'   : 10}
plt.rc('font', **font)

### Difference welfare

* Comparision 1: $DW_{1}:= W_{Baseline} - W_{HLMW}$
    * Baseline economy: Price dispersion + perfectly competitive banks 
    * HLMW: Price dispersion
    * This shows the welfare gains/losses by isolating effects of **banking** only
    
    
* Comparision 2: $DW_{2}:= W_{BCW} -  W_{Baseline}$
    * Baseline economy: Price dispersion + perfectly competitive banks 
    * BCW: Competitive pricing + perfectly competitive banks
    * This shows the welfare gains/losses by isolating effects of **price dispersion** only
    
    
* Comparision 3: $DW_{3}:= W_{BCW} - W_{HLMW}$
    * HLMW: Price dispersion
    * BCW: Competitive pricing + perfectly competitive banks
    * This shows the welfare gains/losses by having both effects of **price dispersion** and **banking**

In [ ]:
#plt.style.use(style='default')
W = result['allocation_grid']['w_star']
W_hlmw = result_hlmw['allocation_grid']['w_star']
DW = W - W_hlmw # Baseline vs. HLMW
plt.figure(facecolor='white')
plt.plot(i_grid*100, DW, label='Baseline vs. HLMW',color='r')
plt.axhline(y=0,color="black", linestyle='--',linewidth=1.0)
plt.ylabel("DW")
plt.xlabel("Nominal interest rate (%)")
plt.legend()
plt.savefig(f"DW_s{model.σ_DM}_ub{model.Ubar_CM}_n{model.n}_α_1{model.α_1}_Tol_1{model_hlmw.Tol}.png")
plt.show()
